In [6]:
import os
import wget

In [7]:
# set data paths  
os.chdir('..')
print('The project path is:', os.getcwd())

data_path           = 'data/'
raw_data_path       = data_path + "01_raw/"
init_dataset_path   = data_path + "02_intermediate/"
url                 = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' 
download_output     = raw_data_path + "/dataset_compressed.tar.gz"

The project path is: /Users/steeve_laquitaine/Desktop/CodeHub/nlp_txt_similarity


In [3]:
# download
wget.download(url, download_output)

NameError: name 'wget' is not defined

In [ ]:
def decompress_dataset(data_file, output_folder):
    # decompress dataset file in dataset/ folder
    tic = time.time()
    tar = tarfile.open(data_file)
    tar.extractall(path=output_folder)
    tar.close()
    toc = time.time()
    print(toc - tic)

In [ ]:
decompress_dataset(download_output, raw_data_path)

In [ ]:
def get_texts(path, CLASSES):
    texts, labels = [], []
    for idx, label in enumerate(CLASSES):
        for fname in (path / label).glob('*.*'):
            texts.append(fname.open('r', encoding='utf-8').read())
            labels.append(idx)
    return np.array(texts), np.array(labels)

def extract_transform_load_dataset(raw_data_path, output_path, timeit=True):
  
    tic = time.time()
    
    # 1 - Init variables
    BOS = 'xbos'  # beginning-of-sentence tag
    FLD = 'xfld'  # data field tag
    CLASSES = ['neg', 'pos', 'unsup']
    col_names = ['sentiment', 'review']
    np.random.seed(42)
    
    # 2 - Build output folders
    PATH = Path(raw_data_path + 'aclImdb/')
    CLAS_PATH = Path(output_path)

    # 3 - Processing and storing train dataset
    print(PATH)
    trn_texts, trn_labels = get_texts(PATH / 'train', CLASSES)
    print(len(trn_texts))
    print(len(trn_labels))
    #trn_idx = np.random.permutation(len(trn_texts))
    #trn_texts = trn_texts[trn_idx]
    #trn_labels = trn_labels[trn_idx]
    df_trn = pd.DataFrame({'review': trn_texts, 'sentiment': trn_labels}, columns=col_names)
    df_trn[df_trn['sentiment'] != 2].to_csv(CLAS_PATH / 'train.csv', index=False)
    
    # 4 - Processing and storing evaluation dataset
    val_texts, val_labels = get_texts(PATH / 'test', CLASSES)
    #val_idx = np.random.permutation(len(val_texts))
    #val_texts = val_texts[val_idx]
    #val_labels = val_labels[val_idx]
    df_val = pd.DataFrame({'review': val_texts, 'sentiment': val_labels}, columns=col_names)
    df_val.to_csv(CLAS_PATH / 'test.csv', index=False)
    
    # 5 - Store the classes
    (CLAS_PATH / 'classes.txt').open('w', encoding='utf-8').writelines(f'{o}\n' for o in CLASSES)

    toc = time.time()
    print(np.round(toc - tic, 2), ' sec')

In [ ]:
extract_transform_load_dataset(raw_data_path, init_dataset_path, timeit = True)

In [ ]:
def load_dataset(train_path, test_path, sample=5000):
    
    tic = time.time()

    # TRAIN
    train_dataset = pd.read_csv(train_path).sample(n=sample)

    # preview data
    print(train_dataset.head())

    # build train and test datasets
    train_reviews = np.array(train_dataset['review'])
    train_sentiments = np.array(train_dataset['sentiment'])

    # TEST
    test_dataset = pd.read_csv(test_path).sample(n=sample)
    test_reviews = np.array(test_dataset['review'])
    test_sentiments = np.array(test_dataset['sentiment'])
    print('(load_dataset)', time.time() - tic)

    toc = time.time()
    print(('Completed'), toc - tic)

    return train_sentiments, test_sentiments, train_reviews, test_reviews

In [ ]:
X_train, X_test, Y_train, Y_test = load_dataset(train_path, test_path, sample=5000)
